## Comunicación con instrumentos - Laboratorio 3

Esta notebook tiene las instrucciones básicas de comunicación con un osciloscopio y un generador de funciones. Encontrarán:

1. Manejo del osciloscopio
   
    a. Seteo de escalas de la pantalla
    
    b.  Adquisición de la pantalla. 
  

3. Manejo del generador

    a. Seteo de la frecuencia y de la amplitud


4. Realizar un barrido en frecuencia, adquirir los datos con el osciloscopio y guardarlos.

5. Elegir una frecuencia, setear escalas del osciloscopio, adquirir y guardar.

Enlaces útiles:

a. [github de Hernán Grecco - Diego Shalom (DF-UBA)](https://github.com/hgrecco/labosdf/tree/master)

b. [github de Marcelo Luda (DF-UBA)](https://marceluda.github.io/python-para-fisicos/tuto/labo2/05_instrumentacion/) 

**Osciloscopio Tektronix. Manuales:**

c1. [Manual de operación](https://download.tek.com/manual/TBS1000B-User-Manual-077088602-RevA.pdf)

c2. [Manual de programación](https://download.tek.com/manual/TBS1000-B-EDU-TDS2000-B-C-TDS1000-B-C-EDU-TDS200-TPS2000-Programmer_EN-US-RevA.pdf)
    
**Generador Tektronix AFG1022. Manuales:** 

d1. [Manual de operación](https://download.tek.com/manual/AFG1000-Quick-Start-User-Manual-EN-077113001.pdf)

d2. [Manual de programación](https://download.tek.com/manual/AFG1000-Programmer-Manual-EN-077112901(20160719)-RevA.pdf)

**Generadores Siglent 1050 - 1010 y 1062. Manual:**

e1. [Manual de programación](https://siglentna.com/USA_website_2014/Documents/Program_Material/SDG_ProgrammingGuide_PG_E03B.pdf)

## Control de los instrumentos y adquisición de una pantalla

In [ ]:
import pyvisa as visa
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd

In [ ]:
# Abro el resource manager
rm = visa.ResourceManager()
# Me fijo que instrumentos tengo conectados
print(rm.list_resources())

In [ ]:
# Nombro cada instrumento
resource_name_osc='USB::0x0699::0x0368::C033542::INSTR'  #Aqui deben colocar el nombre que les figure
#Guardo el instrumento en una variable
osci = rm.open_resource(resource_name_osc)
#Le pido a los instrumentos que se identifiquen
print(osci.query('*IDN?'))

#----------------------------------------------------------------------
fungen = rm.open_resource('USB::0xF4ED::0xEE3A::SDG000::INSTR')

# Con el método 'query()' podemos enviar instrucciones QUE TIENEN RESPUESTA
# Por ejemplo, la instrucción que nos informa el nombre del instrumento
# al que nos conectamos
#print(fungen.query('*IDN?'))

**1.a. Seteo de la escala vertical y horizontal de la pantalla del oscilo**

In [ ]:
# Le preguntamos al osciloscopio la escala vertical del canal 1
print("La escala vertical es: ", osci.query("CH1:SCAle?"))
# Cambiamos la escala vertical:
#osci.write("CH1:SCAle 0.25")

# Preguntamos la escala horizontal
print("La escala horizontal es: ", osci.query("HORizontal:MAIn:SCAle?"))
# Cambiamos la escala horizontal
#osci.write("HORizontal:MAIn:SCAle 0.001")

**1.b. Adquisición de una pantalla (sin guardar los datos)**

**Nota**:Si hay error, verificar que esten prendidos los dos canales del osciloscopio.

In [ ]:
osci.write('autoset EXECUTE') # autoset necesario 
time.sleep(5) #Esperar hasta que autosetee las escalas antes de mandarle otra instruccion

#Prendemos ambos canales del osciloscopio
osci.write('SEL:CH1 ON')
osci.write('SEL:CH2 ON')

# io config, modo de transmision
osci.write('DAT:ENC RPB')
osci.write('DAT:WID 1')

#Canal 1
osci.write('DAT:SOU CH1') 
          # DATa:SOUrce <wfm> \ <wfm> puede ser CH<x>, MATH, REF<x> o FFT

xze1, xin1, yze1, ymu1, yoff1 = osci.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFF?;', separator=';')
data1 = osci.query_binary_values('CURV?', datatype='B', container=np.array)

#Canal 2
osci.write('DAT:SOU CH2') 

xze2, xin2, yze2, ymu2, yoff2 = osci.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFF?;', separator=';')
data2 = osci.query_binary_values('CURV?', datatype='B', container=np.array)


#Conversion a voltaje
tiempo = xze1 + np.arange(len(data1)) * xin1 #tiempo de ch1 y ch2 son iguales
data1v = (data1 - yoff1) * ymu1 + yze1
data2v = (data2 - yoff2) * ymu2 + yze2

plt.figure()
plt.plot(tiempo, data1v, label="Ch1")
plt.plot(tiempo, data2v, label="Ch2")
plt.legend(loc=1)
plt.xlabel('Tiempo [s]')
plt.ylabel('Voltaje [V]')

**2.a. Control de la frecuencia y amplitud del generador**

In [ ]:
######### TEKTRONIX ############
# fungen.write('SOURCE1:FREQUENCY 1kHz') #Hz; kHz; MHz
# fungen.write("SOURCE1:FUNC RAMP")
# fungen.write('SOUR1:VOLT:OFFS 0mV') #Seteo del offset
# fungen.write('SOURce1:VOLTage 0.5Vpp') #Seteo de la amplitud pico a pico
#time.sleep(1)
# fungen.write('SOURCE2:FREQUENCY 100') #Por default las unidades son Hz;

#Si le queremos preguntar la amplitud del canal 1
#print(fungen.query('SOUR1:VOLT?')) #Version reducida del comando

######### SIGLENT ############
fungen.write('C1:BSWV FRQ,2000')
# fungen.write('C1:BSWV AMP,3') 
time.sleep(1)
fungen.write('C2:BSWV FRQ,2000')

## 3. Barrido automático frecuencia y adquisición

**Nota 1**: Para el correcto uso del autoset del osciloscopio, es importante que la señal que ustedes quieren medir sea la fuente del trigger. La fuente del trigger se puede automatizar (ejercicio).

**Nota 2**: Si no tienen una señal en el segundo canal del oscilo, prestar atencion que el autoset no apague el canal durante el barrido (siempre deben estar ambos canales prendidos). 

**Nota 3**: Para frecuencias menores a 20Hz estamos seteando manualmente la escala horizontal. Para frecuencias mayores usamos el autoset del osciloscopio. Prestarle atencion a los tiempos de espera, para que el oscilo llegue a terminar de procesar la instruccion enviada. 

In [ ]:
osci.write('DAT:ENC RPB')
osci.write('DAT:WID 1')

# Creamos listas vacias para guardar los datos
frecuencias_lista = []
tiempos = []
datach1 = []
datach2 = []
deltaVCH1 = []
deltaVCH2 = []

#frecuencias del barrido
frecuencias = np.logspace(1,5,5)

for freq in frecuencias:    
    ############# TEKTRONIX ###############
#     fungen.write('SOURCE1:FREQUENCY {:f}'.format(freq))
    
    ############# SIGLENT ###############
    fungen.write('C1:BSWV FRQ,{:f}'.format(freq))
    print('Comando enviado: ' + 'FREQ {:f}'.format(freq)) 
        
    ############# Osci ###############
    if freq < 20:
        osci.write("HORizontal:MAIn:SCAle 20E-3")
    else:
        time.sleep(10) #Esperar a que el oscilo reciba la nueva señal, antes de pedirle que calcule el autoset 
        osci.write('autoset EXECUTE') # autoset necesario
        # osci.write("AUTORANGE:STATE OFF")

    time.sleep(5)
    
    #Prendemos ambos canales del osciloscopio
    osci.write('SEL:CH1 ON')
    osci.write('SEL:CH2 ON')
    time.sleep(0.5)
    
    #Canal 1
    osci.write('DAT:SOU CH1') 
    xze1, xin1, yze1, ymu1, yoff1 = osci.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFF?;', separator=';')
    data1 = osci.query_binary_values('CURV?', datatype='B', container=np.array)

    #Canal 2
    osci.write('DAT:SOU CH2') 
    xze2, xin2, yze2, ymu2, yoff2 = osci.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFF?;', separator=';')
    data2 = osci.query_binary_values('CURV?', datatype='B', container=np.array)

    #Conversion a voltaje
    tiempo = xze1 + np.arange(len(data1)) * xin1 #tiempo de ch1 y ch2 son iguales
    data1v = (data1 - yoff1) * ymu1 + yze1
    data2v = (data2 - yoff2) * ymu2 + yze2
    
    #Guardamos lo medido
    frecuencias_lista.extend([freq] * len(tiempo))
    tiempos.extend(tiempo)
    datach1.extend(data1v)
    datach2.extend(data2v)
    deltaVCH1.extend([ymu1] * len(tiempo))
    deltaVCH2.extend([ymu2] * len(tiempo)) 
    
    #Graficamos para ir viendo
    plt.plot(tiempo, data1v, label=str(freq)+"Hz")
    plt.plot(tiempo, data2v, label="CH2")
    plt.legend(loc=1)
    plt.show()
    
# Create DataFrame in one step
df = pd.DataFrame({
    "Frecuencias": frecuencias_lista,
    "Tiempo": tiempos,
    "VoltajeCH1": datach1,
    "VoltajeCH2": datach2, 
    "ResolucionVCH1": deltaVCH1, 
    "ResolucionVCH2": deltaVCH2
})

#Guardado
save_folder = 'C:/Users/LSD/Downloads/'
df.to_csv(save_folder+"data_test.csv", index=False, header=True)

**Levantar los datos**

In [ ]:
#Levantar los datos
df_ff = pd.read_csv(save_folder+"data_test.csv", index_col=["Frecuencias", "Tiempo"])

df_ff.head()

In [ ]:
#Graficar los valores de una adquisicion
df_100 = df_ff.loc[100] #Buscamos los datos de 100Hz
df_100.head()

tiempo = df_100.index
v1 = df_100["VoltajeCH1"].values

plt.figure(figsize=(5,3))
plt.plot(tiempo, v1)
plt.show()

## 4. Guardado de datos para una frecuencia

In [ ]:
#Prendemos ambos canales del osciloscopio
osci.write('SEL:CH1 ON')
osci.write('SEL:CH2 ON')
time.sleep(0.5)

#################### Seteo de la frecuencia ##################
freq = 100
#     fungen.write('SOURCE1:FREQUENCY {:f}'.format(freq)) #Si tenes tektronix
fungen.write('C1:BSWV FRQ,{:f}'.format(freq)) #Si tenes SIGLENT

#################### Setear las escalas del oscilo #############
osci.write("HORizontal:MAIn:SCAle 4E-3") #Ver para cada frecuencia la mejor escala
osci.write("CH1:SCAle 2V") #Ver para cada frecuencia la mejor escala
osci.write("CH2:SCAle 2V") #Ver para cada frecuencia la mejor escala

#################### Adquisicion de datos ##################
# io config, modo de transmision
osci.write('DAT:ENC RPB')
osci.write('DAT:WID 1')

#Canal 1
osci.write('DAT:SOU CH1') 
          # DATa:SOUrce <wfm> \ <wfm> puede ser CH<x>, MATH, REF<x> o FFT

xze1, xin1, yze1, ymu1, yoff1 = osci.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFF?;', separator=';')
data1 = osci.query_binary_values('CURV?', datatype='B', container=np.array)

#Canal 2
osci.write('DAT:SOU CH2') 

xze2, xin2, yze2, ymu2, yoff2 = osci.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFF?;', separator=';')
data2 = osci.query_binary_values('CURV?', datatype='B', container=np.array)

#Conversion a voltaje
tiempo = xze1 + np.arange(len(data1)) * xin1 #tiempo de ch1 y ch2 son iguales
data1v = (data1 - yoff1) * ymu1 + yze1
data2v = (data2 - yoff2) * ymu2 + yze2

############# GUARDADO DE DATOS #############
#los acomodamos en un dataframe de pandas
df = pd.DataFrame({'Tiempo':tiempo,'VoltajeCH1':data1v,'VoltajeCH2':data2v})
#si queremos guardar la frecuencia (se agrega una columna extra)
df["Frecuencia"] = freq

#si queremos guardar la resolucion vertical
df["ResolucionVCH1"] = ymu1
df["ResolucionVCH2"] = ymu2

#Aca podemos decir que la frecuencia y el tiempo sean indices
df.set_index(["Frecuencia", "Tiempo"])
#Ahora lo guardamos
save_folder = 'C:/Users/LSD/Downloads/'
df.to_csv(save_folder+"data_frecuencia_"+str(freq)+".csv", index=False, header=True)

plt.figure()
plt.plot(tiempo, data1v, label="Ch1")
plt.plot(tiempo, data2v, label="Ch2")
plt.legend(loc=1)
plt.xlabel('Tiempo [s]')
plt.ylabel('Voltaje [V]')
plt.show()